In [3]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

     |████████████████████████████████| 77 kB 4.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf 

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers # 딥러닝 모델을 위한 라이브러리들

from sklearn.preprocessing import MultiLabelBinarizer # label 인코딩을 위한 라이브러리

from google.colab import files  #csv 파일 colab 상으로 업로드를 위함

In [5]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터 파일 업로드


Saving lunge_dataset.csv to lunge_dataset.csv


In [6]:
import io
data = pd.read_csv(io.BytesIO(csvFile['lunge_dataset.csv']))
data.describe()

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,img_height,img_width,pose_score
count,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000,3593.000000
mean,125.647258,52.626214,0.603521,128.285170,50.698918,0.539946,125.348314,50.970389,0.303186,132.438425,49.155375,0.597191,129.826210,49.033271,0.209920,135.114871,69.040654,0.785938,132.638967,67.851146,0.728137,136.032400,102.931333,0.710208,128.592495,101.764039,0.608559,128.303734,124.793593,0.743139,123.889947,123.850362,0.652840,134.234912,124.102741,0.782713,131.607801,123.881948,0.767109,129.256554,157.366942,0.739673,127.416128,157.986072,0.718072,135.056054,191.660105,0.611146,125.882367,191.310067,0.604312,247.154189,248.398553,0.629742
std,65.905846,47.696374,0.374979,67.617182,47.971360,0.384951,67.178605,48.530000,0.283243,66.904964,49.713672,0.368809,65.920222,48.486672,0.217011,68.794831,51.676708,0.318915,64.794308,50.963244,0.312103,68.862069,55.438850,0.309432,63.597378,55.541480,0.273176,67.078513,57.411166,0.307417,67.081034,58.811116,0.273974,68.063091,62.770255,0.316002,65.035900,62.514450,0.309360,68.420147,71.315660,0.289332,64.470455,69.544812,0.284831,68.299546,84.747402,0.293662,66.272301,84.774753,0.281892,86.671873,100.855706,0.248933
min,-10.266961,-13.311756,0.000528,-7.899345,-10.793723,0.000737,-8.486717,-13.561060,0.000608,-7.927922,-16.848637,0.000386,-7.803750,-17.679083,0.000610,-7.514142,-15.425010,0.000674,-11.738182,-14.129022,0.000594,-8.927567,-6.215543,0.000537,-10.103379,-8.580183,0.000576,-9.849440,-5.287575,0.000613,-5.326314,-5.101714,0.000805,-6.494373,-6.097191,0.000802,-6.254012,-7.448987,0.000718,-10.041183,-6.990811,0.001133,-7.021356,-5.702689,0.000901,-6.753867,-0.903168,0.000942,-7.085725,-0.151686,0.000773,199.000000,75.000000,0.001182
25%,86.309809,15.218757,0.212194,88.496504,13.245644,0.105095,86.498243,13.189060,0.056498,94.213516,14.418104,0.210844,95.441537,14.344104,0.055953,94.953539,31.475383,0.732933,98.545126,30.787693,0.575600,95.792079,59.855303,0.544073,94.397707,58.315075,0.478940,88.227156,87.026990,0.660831,83.928036,85.837823,0.554181,94.968036,80.359777,0.814586,96.783137,80.430364,0.773001,91.871462,121.688665,0.717914,92.741628,121.152203,0.667928,85.866617,158.297105,0.410848,70.101535,157.932952,0.418164,224.000000,224.000000,0.554051
50%,112.807385,44.546719,0.768090,115.553650,41.327777,0.631925,111.071071,41.820636,0.220412,121.920102,38.935628,0.755917,115.267751,39.188794,0.137574,122.267565,62.418440,0.955530,116.283220,61.976132,0.882223,126.779004,100.299628,0.867189,114.536625,97.848652,0.677234,120.837165,120.924822,0.887337,111.446063,117.528768,0.733568,121.914823,118.701081,0.931170,116.823499,117.553774,0.912865,122.843127,138.137367,0.850714,117.231966,138.060429,0.811528,143.991296,178.662427,0.689005,128.156698,179.076320,0.732248,224.000000,224.000000,0.725555
75%,150

## 데이터 셋 설명
자체 수집한 런지 사진 데이터를 PoseNet 모델을 통해 <br>
0.코   
1.왼쪽 눈  
2.오른쪽 눈  
3.왼쪽 귀  
4.오른쪽 귀  
5.왼쪽 어깨  
6.오른쪽 어깨  
7.왼쪽 팔꿈치  
8.오른쪽 팔꿈치  
9.왼쪽 손목  
10.오른쪽 손목  
11.왼쪽 골반 부위  
12.오른쪽 골반 부위  
13.왼쪽 무릎  
14.오른쪽 무릎  
15.왼쪽 발목  
16.오른쪽 발목  

각각의 x,y 좌표와 신뢰도점수, 이미지 크기와 ,전체 신뢰도 점수를 구해 데이터 셋으로 만들었습니다.  

[출처) 포즈예측 | Tensorflow Lite](https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ko)




In [7]:
for i in range(17):
  str_i = str(i)
  data[str_i+'_position_x'] = data[str_i+'_position_x'] / data['img_width']
  data[str_i+'_position_y'] = data[str_i+'_position_y'] / data['img_height']

# 이미지 크기로 각 x,y 좌표를 나누어 이미지 크기에 구애 받지 않는 데이터셋으로 변환
label = data['label']
features = data.drop(['label'],axis=1).drop(['img_width'],axis=1).drop(['img_height'],axis=1) # label과 feature 분리

In [8]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, label, test_size=0.3)

In [9]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(train_x.keys())]),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax')
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [10]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________


In [11]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(",")) #Label One-Hot Encoding

In [12]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3)  #Train set과 Test set 구분

In [13]:
from sklearn.utils import validation
model.fit(train_x,train_y, epochs = 30)

Epoch 1/30
79/79 [==============================] - 1s 2ms/step - loss: 0.5402 - acc: 0.8258
Epoch 2/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1866 - acc: 0.9396
Epoch 3/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1473 - acc: 0.9527
Epoch 4/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1367 - acc: 0.9571
Epoch 5/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1201 - acc: 0.9630
Epoch 6/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1200 - acc: 0.9610
Epoch 7/30
79/79 [==============================] - 0s 2ms/step - loss: 0.0995 - acc: 0.9650
Epoch 8/30
79/79 [==============================] - 0s 2ms/step - loss: 0.1081 - acc: 0.9630
Epoch 9/30
79/79 [==============================] - 0s 2ms/step - loss: 0.0889 - acc: 0.9694
Epoch 10/30
79/79 [==============================] - 0s 2ms/step - loss: 0.0759 - acc: 0.9761
Epoch 11/30
79/79 [==============================] - 0s 2ms/step - lo

In [14]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증
mlb.classes_ # 출력 형태

34/34 [==============================] - 0s 1ms/step - loss: 0.0541 - acc: 0.9852


array(['lunge', 'nothing', 'standingside'], dtype=object)

In [15]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

Save-Model


In [16]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./lunge_model.json")


In [17]:
model.save('lunge_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________
